# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     Rogelio Adrian Arroyo Valencia   |
| **Fecha**      17/02/2025|   |
| **Expediente**746926 |   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Si, al revisar que nuestro modelo muestra buenos métricos en train y test podemos utilizar nuestros datos completos

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [71]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [73]:
data = pd.read_excel("Motor Trend Car Road Tests.xlsx")
data_filtered = data.drop(columns=['model'])

In [97]:
X = data_filtered.drop(columns=['mpg'])
y = data_filtered['mpg']
# Aplicar escalamiento a las características
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [101]:
mse_values = []
#LeaveOneOut Cross Validation DE MANERA manual
for i in range(len(data_filtered)): 
    X_train = X.drop(index=i).values
    # Aplicar escalamiento a las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    y_train = y.drop(index=i).values
    X_test = X.iloc[i, :].values.reshape(1, -1)
    X_test_scaled = scaler.transform(X_test)
    y_test = y.iloc[i].reshape(-1, 1)

    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)
    print(f'Combinación {i+1}: MSE = {mse}')

Combinación 1: MSE = 5.258855796629049
Combinación 2: MSE = 2.453996713086807
Combinación 3: MSE = 20.550493416882723
Combinación 4: MSE = 0.0443291071396446
Combinación 5: MSE = 1.5811555159760176
Combinación 6: MSE = 10.11862548887538
Combinación 7: MSE = 0.0163740415484736
Combinación 8: MSE = 8.081250710389478
Combinación 9: MSE = 39.47018347964978
Combinación 10: MSE = 0.7706327722901085
Combinación 11: MSE = 4.955694234215832
Combinación 12: MSE = 10.224730198354477
Combinación 13: MSE = 4.430129331402683
Combinación 14: MSE = 0.4878127712478593
Combinación 15: MSE = 6.8238445758093995
Combinación 16: MSE = 0.6027506975546868
Combinación 17: MSE = 36.81017860995889
Combinación 18: MSE = 31.759855581210395
Combinación 19: MSE = 1.063227541805811
Combinación 20: MSE = 32.7132861870273
Combinación 21: MSE = 14.307173682712742
Combinación 22: MSE = 3.405354649998888
Combinación 23: MSE = 9.408007649079861
Combinación 24: MSE = 0.0001035003980367482
Combinación 25: MSE = 9.96243953454

In [109]:
mse_std = np.std(mse_values)
mse_std

17.06739987188858

In [107]:
mse_mean = np.mean(mse_values)
mse_mean

12.181558006901941

Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3